In [1]:
!pip install beautifulsoup4


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

##request.get not working

In [86]:
def extract(jobs):
    headers = {'User-Agent':'Mozilla/5.0'}
    url = f'https://www.monsterindia.com/search/sales-jobs?skills=sales&filter=true'
    r = requests.get(url,headers)
    print(r.status_code)

In [88]:
extract(0)

403


## an extract function to call the link and get the soup object

In [2]:
def extract(link):
    from urllib.request import Request, urlopen
    url=link
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})

    web_byte = urlopen(req).read()
    webpage = web_byte.decode('utf-8')
    soup = BeautifulSoup(webpage,'html.parser')
    return soup

## transforming the content into title, company, salary, experience and job description

In [3]:
def transform(soup):
    divs = soup.find_all('div', class_='card-apply-content')
    for item in divs:
        title = item.find('div', class_ = 'job-tittle')
        job_title = title.find('a').text.strip()
        title_list.append(job_title)
        company = title.find('span',class_='company-name').text.strip()
        company_list.append(company)
        search_tag = title.find('div',class_ = 'searctag row')
        experience = search_tag.find('div', class_='exp col-xxs-12 col-sm-3 text-ellipsis').text.strip()
        experience_list.append(experience)
        try:
            salary = search_tag.find('div', class_ ='package col-xxs-12 col-sm-4 text-ellipsis hidden-sm').text.strip()
        except:
            salary = 'not specified'
        salary_list.append(salary)
        description_link_tag = title.find('h3', class_='medium')
        description_link = description_link_tag.a['href']
        bs = extract(description_link)
        try:
            job_description = bs.find('div', class_='jd-text').text.strip()
        except:
            job_description = 'job description is not available for this job'
        if salary =='not specified':
            try:
                salary = bs.find('div', class_ ='package col-xxs-12 col-sm-4 text-ellipsis hidden-sm').text.strip()
            except:
                salary = 'not specified'
        description_list.append(job_description)
    return


In [24]:
title_list = []
company_list = []
experience_list = []
salary_list = []
description_list = []

## calling the monster.com by categorising using different skill and extracting jobs in first ten pages.

In [26]:
skills = ['accounting-jobs','analytics-jobs','animation-jobs','architecture-jobs','banking-jobs','java-jobs','data-science-jobs','marketing-jobs','networking-jobs','online-marketing-jobs','online-marketing-jobs']

for skill in skills:
    title_list.clear()
    company_list.clear()
    experience_list.clear()
    salary_list.clear()
    description_list.clear()
    c =extract(f'https://www.monsterindia.com/search/{skill}')
    transform(c)
    i = 0
    for i in range(2,11):
        c =extract(f'https://www.monsterindia.com/search/{skill}-{i}')
        transform(c)

    header = ["SN","Title", "Company", "Experience","Salary","Description"]
    x = 0

    with open(f"{skill}.csv", 'w',encoding='UTF8',newline='') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        while x < len(title_list):
            writer.writerow([x,title_list[x],company_list[x],experience_list[x],salary_list[x],description_list[x]])
            x += 1

In [ ]:
df = pd.read_csv('accounting-jobs.csv')

In [97]:
df.head()

,Unnamed: 0,SN,Title,Company,Experience,Salary,Description
0,0,0,FUND ACCOUNTING ANALYST,Citi,Not Specified,Not Specified,The Fund Accounting Analyst is an senior level...
1,1,1,AVP - FBR Accounting,Citi,8-10 years,Not Specified,The Finance Reporting Manager is an intermedia...
2,2,2,Finance and Accounting,Tata Group,1-6 years,Not Specified,BPO3 (Team Member Team LeaderNagpur.Graduate w...
3,3,3,Financial Accounting Associate,Wells Fargo,3-6 years,Not Specified,Job Description :About this role:Wells Fargo i...
4,4,4,C04- Fund Accounting,Citi,1-3 years,Not Specified,The Fund Accounting Specialist is an entry lev...


In [92]:
import re

## Finding words like Salary, CTC in description, if found then generating a different csv file to store all the information.

In [ ]:
count = 0
df_length = len(df['Salary'])
i = 0
sentence = []
sentence_string = ''
while i < 249:
    sentence.clear()
    if df['Salary'][i]=='Not Specified':
        text = str(df['Description'][i])
        print(text)
        index = text.find('salary') or text.find('Salary') or text.find('ctc') or text.find('CTC')
        if index:
            if text[index] =='s':
                sentence.append(re.findall(r"([^.]*?salary[^.]*\.)",text))
            if text[index] =='S':
                sentence.append(re.findall(r"([^.]*?Salary[^.]*\.)",text))
            if text[index] == 'C':
                sentence.append(re.findall(r"([^.]*?CTC[^.]*\.)",text))

            sentence_string.join(sentence)
                
            salary_text = re.findall("[\$0-9,\. ]*-[\$0-9,\. ]*", sentence_string)

            if not salary_text :
                continue
            else:
                count += 1
                df['Salary'][i]= salary_text
    i += 1            

    
            
        

    

In [82]:
print(count)

0


In [73]:
if count > 0:
    df.to_csv("updated_accounting-jobs.csv")